In [69]:
# 코랩에 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import warnings
# warnings.filterwarnings(action = "ignore")

# 데이터 처리 모듈
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm

# 멀티프로세싱
import multiprocessing as mp
from multiprocessing import Pool

# 척도 변환 모듈(표준화, 정규화, 로버스트 정규화, 원핫인코딩(범주형에서 더미변수로 변환) 등)
from sklearn.preprocessing import *

# 결측값 관측 모듈
import missingno as msno

# 시각화 모듈
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# 배열, 행렬 연산 모듈
import numpy as np

# 데이터 분할 모듈(훈련용 / 검증용 / 시험용)
from sklearn.model_selection import train_test_split

# 통계적 가설검정 및 계량화 모듈
# https://youtu.be/FtWEZw3kUho
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.interpolate import UnivariateSpline

# 타입 어노테이션 모듈
from typing import *

# 시간 관련 모듈
from tqdm import tqdm
from time import strptime, sleep

# 웹크롤링 / 스크래핑 관련 모듈
import requests
import io
import zipfile
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

# 결측값 대체 및 특징 추출
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA, SparsePCA

# 연관규칙 분석 실행 모듈
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx

# 한글폰트 설정
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
import matplotlib.patches as mpatches

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 주식종목 데이터 로드 모듈
!pip install pykrx
from pykrx import stock
!pip install finance-datareader --upgrade
import FinanceDataReader as fdr

In [3]:
class AprioriRuleswithEvent():

  def __init__(self, file_path: str, DROP_COLUMN_LIST: List, SUBSET = None):

    self.INDEX_NEWS_DATA = pd.read_csv(file_path, index_col = False, na_values = "NaN")
      # [file_path] "/content/drive/MyDrive/AfterLearnerProject/DataArchive/merge_derivative_news_data.csv"
      # [INDEX_PRINCIPAL_COMPONENT_x] 시계열 전처리한 5개 주가지수선형결합 제1주성분
      # [INDEX_PRINCIPAL_COMPONENT_y] 시계열 전처리한 5개 주가지수선형결합 제1주성분의 단위변동률
    self.INDEX_NEWS_DROPNA = self.INDEX_NEWS_DATA.dropna(subset = DROP_COLUMN_LIST)
      # msno.dendrogram()
    self.SUBSET = SUBSET


  def compareEventDatetimewithNewsImportanceScore(self):
    # improtance와 특이 지점을 비교하기 위해 importance값 상위 20개 추출(특이지점과 흡사)
    display( self.INDEX_NEWS_DATA.sort_values(
        by = "IMPORTANCE", ascending = False).head(20)[["IMPORTANCE", "YYYYMMDD", "TAG_LIST"]] )


  def removeBlank(self):
    import re
    self.INDEX_NEWS_DROPNA["TAG_LIST"] = self.INDEX_NEWS_DROPNA["TAG_LIST"].apply(
        lambda tag: re.sub(" ", "", tag))
    self.INDEX_NEWS_DROPNA["ITEM_NAME"] = self.INDEX_NEWS_DROPNA["ITEM_NAME"].apply(lambda tag: re.sub(" ", "", tag))
      # "\s{0,}"
      # re.sub(" +", "" , string)
      # " ".join(string.split())


  def mutateNewColumn(self):
    self.INDEX_NEWS_DROPNA["CHANGE_CATEGORY"] = (
        self.INDEX_NEWS_DROPNA["STABLE_or_DRAMATIC"].astype(str) + "_" + self.INDEX_NEWS_DROPNA["SIGN_CHANGE"].astype(str))
    self.INDEX_NEWS_DROPNA["TAG_SPLIT"] = self.INDEX_NEWS_DROPNA.TAG_LIST.str.split("|")
    self.INDEX_NEWS_DROPNA["ITEM_SPLIT"] = self.INDEX_NEWS_DROPNA.ITEM_NAME.str.split("|")
    print( "\n", Counter(self.INDEX_NEWS_DROPNA["CHANGE_CATEGORY"]) , "\n", Counter(self.INDEX_NEWS_DROPNA["YYYYMMDD"]) )


  def makeSubDataset_loadData(self, date_column: str, date_time: str):

    INDEX_NEWS_DATETIME = self.INDEX_NEWS_DROPNA[ self.INDEX_NEWS_DROPNA[date_column] == date_time ]
    self.SUBSET = INDEX_NEWS_DATETIME[["CHANGE_CATEGORY", "SUBCATEGORY"]].values
      # [column] "ITEM_NAME", "SUBCATEGORY"
    return self.SUBSET, INDEX_NEWS_DATETIME
      # [date_column] YYYYMMDD


  # Apriori Analysis
  def find_frequent_TAGsets(self, transaction_itemset: pd.Series, transaction_tagset: pd.Series,
                            drop_tagset_list: List,
                            minimum_support_threshold: float,
                            minimum_confidence_threshold: float):

      transaction_item_encoder = TransactionEncoder()
      transaction_tag_encoder = TransactionEncoder()
      transaction_item_array = transaction_item_encoder.fit_transform(transaction_itemset)
      transaction_tag_array = transaction_tag_encoder.fit_transform(transaction_tagset)

      transaction_itemset_dataframe = pd.DataFrame(
          data = transaction_item_array, columns = transaction_item_encoder.columns_)
      transaction_tagset_dataframe = pd.DataFrame(
          data = transaction_tag_array, columns = transaction_tag_encoder.columns_)
      transaction_dataframe = pd.concat(
          [transaction_itemset_dataframe, transaction_tagset_dataframe],
          axis = 1, ignore_index = False)

    # 이 지지도 이하의 규칙들은 배제하는 threshold (임계점 or 한계치)를 설정
      frequent_TAG_sets = apriori(
          transaction_dataframe,
          max_len = 2,
          min_support = minimum_support_threshold,
          use_colnames = True )

    # apriori에 최소 지지도를 적용한 빈발원소집합, frequent_TAG_sets에서
    # [metric][confidence] 품목 사이의 연관정 정도를 측정하기 위해 사용되는 지표
    # [metric][lift] 두 사건이 서로 독립일 때와 비교해 얼마나 발생했는가에 대해 비율
    # print(apriori.__doc__)
      apriori_rules = association_rules(
          frequent_TAG_sets, metric = "confidence", min_threshold = minimum_confidence_threshold)
      return apriori_rules, transaction_item_encoder.columns_, transaction_tag_encoder.columns_


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
financialApriori = AprioriRuleswithEvent(
    file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/merge_derivative_news_data.csv",
    DROP_COLUMN_LIST = ["TAG_LIST", "ITEM_NAME"]
)

financialApriori.compareEventDatetimewithNewsImportanceScore()
financialApriori.removeBlank()
financialApriori.mutateNewColumn()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,IMPORTANCE,YYYYMMDD,TAG_LIST
43463,168.84,2023-02-01,게임| 리니지| 결제| 아이템| 산업| 디아블로| 캐릭터| 장비| 그래픽| PC| ...
42206,146.29,2023-02-01,2차전지| 글로벌| 전기차| 전극| 자동차| 장비| 에너지| 배터리팩| 주행거리| 가전
4243,129.80,2023-01-25,코로나| 호텔| 중국| 면세점| 여행| 관광객| 화장품| 제주| 일본| 여권| 소비
60764,125.75,2023-02-02,우주| 공연| 축구| 스포츠| 경찰| 공기| 발사| 문화| 태풍| 조선| 겨울| 운동
19339,111.14,2023-01-26,가구| 전기| 가스
13673,107.20,2023-01-26,교육| 양육
14055,100.42,2023-01-26,일본| 캐릭터| 도깨비| 캔버스
7666,100.09,2023-01-25,기온| 추위| 바닷물| 난방| 겨울| 청주| 제주
5739,98.79,2023-01-25,제주| 공항| 기온
32676,96.63,2023-01-31,경찰| 테러



 Counter({'DRAMATIC_Not Change': 12281, 'DRAMATIC_Sign Change': 3252}) 
 Counter({'2023-03-15': 2918, '2023-03-10': 2152, '2023-04-12': 1858, '2023-02-01': 1687, '2023-01-31': 1621, '2023-02-02': 1565, '2023-01-26': 1376, '2023-01-25': 1294, '2023-01-27': 1062})


## **2023-01-25** | DRAMATIC_Not Change
(AI)	(인공지능) | (인공지능)	(AI)

(은행)	(금리) | (대출)	(금리)


In [5]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230125, merge_derivative_news_data_230125 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-25")
# pd.set_option("display.max_row", 10)
# Counter(merge_derivative_news_data_230125["SUBCATEGORY"])

apriori_results_230125: List = []
transactions_itemset_230125: List = []
transactions_tagset_230125: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230125["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230125["TAG_SPLIT"],
      drop_tagset_list = [""],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230125.append(apriori_rules)
    transactions_itemset_230125.append(transactions_itemset)
    transactions_tagset_230125.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230125[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230125[1].sort_values("confidence", ascending = False) )
display( apriori_results_230125[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
222,(OLED),(디스플레이),0.006182,0.022411,0.006182,1.000000,44.620690,0.006044,inf,0.983670
43,(바이러스),(LG전자),0.005410,0.016229,0.005410,1.000000,61.619048,0.005322,inf,0.989122
199,(광산),(EV),0.005410,0.013138,0.005410,1.000000,76.117647,0.005339,inf,0.992230
71,(앱스토어),(넥슨게임즈),0.005410,0.008501,0.005410,1.000000,117.636364,0.005364,inf,0.996892
196,(채식주의),(CU),0.006182,0.009274,0.006182,1.000000,107.833333,0.006125,inf,0.996890
...,...,...,...,...,...,...,...,...,...,...
102,(아시아),(메드팩토),0.017002,0.007728,0.006955,0.409091,52.936364,0.006824,1.679230,0.998078
311,(아시아),(헬스케어),0.017002,0.023184,0.006955,0.409091,17.645455,0.006561,1.653073,0.959644
251,(아시아),(네트워크),0.017002,0.011592,0.006955,0.409091,35.290909,0.006758,1.672691,0.988470
228,(PC),(전기차),0.020866,0.044049,0.008501,0.407407,9.248863,0.007582,1.613167,0.910885


In [7]:
import itertools
transactions_itemset_230125 = list( itertools.chain.from_iterable(transactions_itemset_230125) )
transactions_tagset_230125 = list( itertools.chain.from_iterable(transactions_tagset_230125) )
print(len(transactions_itemset_230125), len(transactions_tagset_230125))

1938 1917


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-01-26** | DRAMATIC_Not Change

(대출)	(금리) | (금리, 은행)	(대출)

(무인기)	(북한)	| (북한)	(무인기)

In [8]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230126, merge_derivative_news_data_230126 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-26")

apriori_results_230126: List = []
transactions_itemset_230126: List = []
transactions_tagset_230126: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230126["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230126["TAG_SPLIT"],
      drop_tagset_list = ["톰슨로이터"],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230126.append(apriori_rules)
    transactions_itemset_230126.append(transactions_itemset)
    transactions_tagset_230126.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230126[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230126[1].sort_values("confidence", ascending = False) )
display( apriori_results_230126[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(CU),(BGF리테일),0.005087,0.005814,0.005087,1.000000,172.000000,0.005058,inf,0.999270
304,(코발트),(니켈),0.007267,0.013081,0.007267,1.000000,76.444444,0.007172,inf,0.994143
271,(코나),(글로벌),0.005087,0.170058,0.005087,1.000000,5.880342,0.004222,inf,0.834186
260,(구리),(필터),0.006541,0.014535,0.006541,1.000000,68.800000,0.006446,inf,0.991953
253,(구리),(에너지),0.006541,0.039971,0.006541,1.000000,25.018182,0.006279,inf,0.966350
...,...,...,...,...,...,...,...,...,...,...
175,(ESG),(결제),0.015988,0.022529,0.006541,0.409091,18.158358,0.006180,1.654182,0.960282
41,(NAVER),(현대차),0.014535,0.067587,0.005814,0.400000,5.918280,0.004832,1.554021,0.843289
75,(삼성SDI),(엘앤에프),0.014535,0.018895,0.005814,0.400000,21.169231,0.005539,1.635174,0.966814
284,(도시가스),(난방),0.014535,0.012355,0.005814,0.400000,32.376471,0.005634,1.646076,0.983407


In [10]:
transactions_itemset_230126 = list( itertools.chain.from_iterable(transactions_itemset_230126) )
transactions_tagset_230126 = list( itertools.chain.from_iterable(transactions_tagset_230126) )
print(len(transactions_itemset_230126), len(transactions_tagset_230126))

1956 1947


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-01-27** | DRAMATIC_Not Change
(마스크)	(코로나19) | (코로나19) (마스크)

(전기차)	(중국)	| (전기차)	(글로벌)

(전기차)  (배터리)	| (배터리)	(전기차)

In [11]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230127, merge_derivative_news_data_230127 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-27")

apriori_results_230127: List = []
transactions_itemset_230127: List = []
transactions_tagset_230127: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230127["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230127["TAG_SPLIT"],
      drop_tagset_list = ["톰슨로이터"],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230127.append(apriori_rules)
    transactions_itemset_230127.append(transactions_itemset)
    transactions_tagset_230127.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230127[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230127[1].sort_values("confidence", ascending = False) )
display( apriori_results_230127[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
187,(상사),(LNG),0.005650,0.013183,0.005650,1.0,75.857143,0.005575,inf,0.992424
197,(사운드),(OLED),0.005650,0.036723,0.005650,1.0,27.230769,0.005442,inf,0.968750
165,(B2B),(글로벌),0.005650,0.195857,0.005650,1.0,5.105769,0.004543,inf,0.808712
164,(B2B),(TV),0.005650,0.083804,0.005650,1.0,11.932584,0.005176,inf,0.921402
163,(챗봇),(AI),0.010358,0.051789,0.010358,1.0,19.309091,0.009821,inf,0.958135
...,...,...,...,...,...,...,...,...,...,...
91,(기아),(현대차),0.042373,0.059322,0.016949,0.4,6.742857,0.014436,1.567797,0.889381
351,(인공지능),(지능형로봇),0.014124,0.005650,0.005650,0.4,70.800000,0.005570,1.657250,1.000000
265,(납품),(바이러스),0.014124,0.005650,0.005650,0.4,70.800000,0.005570,1.657250,1.000000
96,(기아),(전기차),0.042373,0.064972,0.016949,0.4,6.156522,0.014196,1.558380,0.874631


In [13]:
transactions_itemset_230127 = list( itertools.chain.from_iterable(transactions_itemset_230127) )
transactions_tagset_230127 = list( itertools.chain.from_iterable(transactions_tagset_230127) )
print(len(transactions_itemset_230127), len(transactions_tagset_230127))

1593 1677


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-01-31** | DRAMATIC_Not Change
(반도체)	(메모리) | (메모리)	(반도체) | (파운드리)	(반도체)

(반도체)	(글로벌) | (일본)	(중국) | (코로나19)	(중국) | (소비)	(중국)

(부동산)	(주택)	| (주택)	(부동산)

In [14]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230131, merge_derivative_news_data_230131 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-31")

apriori_results_230131: List = []
transactions_itemset_230131: List = []
transactions_tagset_230131: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230131["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230131["TAG_SPLIT"],
      drop_tagset_list = ["톰슨로이터"],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230131.append(apriori_rules)
    transactions_itemset_230131.append(transactions_itemset)
    transactions_tagset_230131.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230131[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230131[1].sort_values("confidence", ascending = False) )
display( apriori_results_230131[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
140,(주파수),(5G),0.007403,0.015423,0.007403,1.000000,64.840000,0.007289,inf,0.991920
97,(예스티),(반도체),0.005552,0.088834,0.005552,1.000000,11.256944,0.005059,inf,0.916253
309,(에어컨),(친환경),0.005552,0.036397,0.005552,1.000000,27.474576,0.005350,inf,0.968983
198,(셔터스톡),(글로벌),0.005552,0.122147,0.005552,1.000000,8.186869,0.004874,inf,0.882754
247,(마이크로바이옴),(인도),0.006786,0.009870,0.006786,1.000000,101.312500,0.006719,inf,0.996894
...,...,...,...,...,...,...,...,...,...,...
129,(현대차),(자동차),0.028994,0.044417,0.011721,0.404255,9.101359,0.010433,1.604014,0.916706
139,(5G),(스마트폰),0.015423,0.037014,0.006169,0.400000,10.806667,0.005598,1.604976,0.921679
23,(5G),(LG유플러스),0.015423,0.019741,0.006169,0.400000,20.262500,0.005865,1.633765,0.965539
180,(게임),(플랫폼),0.015423,0.054904,0.006169,0.400000,7.285393,0.005322,1.575159,0.876253


In [16]:
transactions_itemset_230131 = list( itertools.chain.from_iterable(transactions_itemset_230131) )
transactions_tagset_230131 = list( itertools.chain.from_iterable(transactions_tagset_230131) )
print(len(transactions_itemset_230131), len(transactions_tagset_230131))

1845 1944


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-02-01** | DRAMATIC_Sign Change

(중국)	(글로벌) | (리오프닝)	(중국)

(대출)	(금리)

(반도체)	(메모리)

In [17]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230201, merge_derivative_news_data_230201 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-02-01")

apriori_results_230201: List = []
transactions_itemset_230201: List = []
transactions_tagset_230201: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230201["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230201["TAG_SPLIT"],
      drop_tagset_list = ["톰슨로이터"],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230201.append(apriori_rules)
    transactions_itemset_230201.append(transactions_itemset)
    transactions_tagset_230201.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230201[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230201[1].sort_values("confidence", ascending = False) )
display( apriori_results_230201[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
58,(여객),(대한항공),0.008299,0.009484,0.008299,1.000000,105.437500,0.008220,inf,0.998805
215,(아이오닉),(충전),0.005335,0.008892,0.005335,1.000000,112.466667,0.005287,inf,0.996424
106,(완성차),(현대차),0.006520,0.045643,0.006520,1.000000,21.909091,0.006223,inf,0.960621
105,(아이오닉),(현대차),0.005335,0.045643,0.005335,1.000000,21.909091,0.005091,inf,0.959476
99,(현대두산인프라코어),(ESG),0.005928,0.015412,0.005928,1.000000,64.884615,0.005836,inf,0.990459
...,...,...,...,...,...,...,...,...,...,...
235,(치료),(환자),0.016005,0.026082,0.006520,0.407407,15.620370,0.006103,1.643487,0.951205
116,(2차전지),(신규상장),0.016005,0.014819,0.006520,0.407407,27.491852,0.006283,1.662493,0.979299
74,(스마트폰),(삼성전자),0.016005,0.098992,0.006520,0.407407,4.115547,0.004936,1.520451,0.769332
14,(화장품),(LG생활건강),0.014819,0.020747,0.005928,0.400000,19.280000,0.005620,1.632089,0.962395


In [19]:
transactions_itemset_230201 = list( itertools.chain.from_iterable(transactions_itemset_230201) )
transactions_tagset_230201 = list( itertools.chain.from_iterable(transactions_tagset_230201) )
print(len(transactions_itemset_230201), len(transactions_tagset_230201))

2031 2031


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-02-02** | DRAMATIC_Sign Change

(스마트폰)	(갤럭시) | (갤럭시)	(스마트폰) | (카메라)	(갤럭시) | (디지털)	(플랫폼)

(금리)	(중앙은행) | (금융)  (금리) | (채권)	(금리)

(배터리)	(전기차)	|  (전기차)	(배터리)



(전기)	(가스)

In [20]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230202, merge_derivative_news_data_230202 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-02-02")

apriori_results_230202: List = []
transactions_itemset_230202: List = []
transactions_tagset_230202: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230202["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230202["TAG_SPLIT"],
      drop_tagset_list = ["톰슨로이터"],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230202.append(apriori_rules)
    transactions_itemset_230202.append(transactions_itemset)
    transactions_tagset_230202.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230202[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230202[1].sort_values("confidence", ascending = False) )
display( apriori_results_230202[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
238,(면역항암제),(임상),0.005751,0.027476,0.005751,1.000000,36.395349,0.005593,inf,0.978149
218,(세탁기),(냉장고),0.005112,0.007029,0.005112,1.000000,142.272727,0.005076,inf,0.998073
189,(칩셋),(갤럭시),0.005112,0.070927,0.005112,1.000000,14.099099,0.004749,inf,0.933847
266,(스테인리스),(에너지),0.007668,0.023003,0.007668,1.000000,43.472222,0.007491,inf,0.984546
192,(프로세서),(갤럭시),0.010224,0.070927,0.010224,1.000000,14.099099,0.009499,inf,0.938670
...,...,...,...,...,...,...,...,...,...,...
45,(NAVER),(금리),0.026837,0.040895,0.010863,0.404762,9.897693,0.009765,1.611297,0.923757
72,(카카오),(삼성전자),0.036422,0.143770,0.014696,0.403509,2.806628,0.009460,1.435444,0.668031
143,(전기차),(현대차),0.035144,0.032588,0.014058,0.400000,12.274510,0.012912,1.612354,0.951987
145,(신규상장),(2차전지),0.012780,0.016613,0.005112,0.400000,24.076923,0.004900,1.638978,0.970874


In [22]:
transactions_itemset_230202 = list( itertools.chain.from_iterable(transactions_itemset_230202) )
transactions_tagset_230202 = list( itertools.chain.from_iterable(transactions_tagset_230202) )
print(len(transactions_itemset_230202), len(transactions_tagset_230202))

2145 2004


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-03-10** | DRAMATIC_Not Change

(이머징마켓, 아시아)	(미국) | (미국, 이머징마켓)	(아시아) | (중국, 아시아)	(이머징마켓)

(EU, 미국) | (, 소재)	(미국)

(채권)	(금리)

In [23]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230310, merge_derivative_news_data_230310 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-03-10")

apriori_results_230310: List = []
transactions_itemset_230310: List = []
transactions_tagset_230310: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
      transaction_itemset = merge_derivative_news_data_230310["ITEM_SPLIT"],
      transaction_tagset = merge_derivative_news_data_230310["TAG_SPLIT"],
      drop_tagset_list = ["톰슨로이터"],
      minimum_support_threshold = threshold,
      minimum_confidence_threshold = confidence
    )
    apriori_results_230310.append(apriori_rules)
    transactions_itemset_230310.append(transactions_itemset)
    transactions_tagset_230310.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230310[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230310[1].sort_values("confidence", ascending = False) )
display( apriori_results_230310[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
36,(),(톰슨로이터),0.111059,0.294610,0.294610,2.652720,9.004184,0.261890,inf,1.000000
0,(도큐사인),(),0.005112,0.111059,0.005112,1.000000,9.004184,0.004544,inf,0.893508
120,(다우),(미국),0.005112,0.148234,0.005112,1.000000,6.746082,0.004354,inf,0.856142
31,(일자리),(),0.005576,0.111059,0.005576,1.000000,9.004184,0.004957,inf,0.893925
37,(톰슨로이터),(),0.294610,0.111059,0.294610,1.000000,9.004184,0.261890,inf,1.260211
...,...,...,...,...,...,...,...,...,...,...
166,(콘텐츠),(플랫폼),0.012546,0.029275,0.005112,0.407407,13.916520,0.004744,1.638098,0.939936
45,(LG전자),(대여),0.012546,0.009294,0.005112,0.407407,43.837037,0.004995,1.671817,0.989604
86,(제이피모간체이스),(미국),0.012546,0.148234,0.005112,0.407407,2.748404,0.003252,1.437355,0.644235
28,(이머징마켓),(),0.032063,0.111059,0.013011,0.405797,3.653872,0.009450,1.496022,0.750377


In [25]:
transactions_itemset_230310 = list( itertools.chain.from_iterable(transactions_itemset_230310) )
transactions_tagset_230310 = list( itertools.chain.from_iterable(transactions_tagset_230310) )
print(len(transactions_itemset_230310), len(transactions_tagset_230310))

2229 2181


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **2023-04-12** | DRAMATIC_Not Change
(미국, 아시아)	(이머징마켓) | (미국, 이머징마켓)	(아시아) | (아시아, 이머징마켓)	(미국)
(소재)	(미국) | (헬스케어)	(미국)	| (미세먼지)	(황사)
(중국, 이머징마켓)	(아시아) | (EU, 아시아)	(이머징마켓)

In [26]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230412, merge_derivative_news_data_230412 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-04-12")

apriori_results_230412: List = []
transactions_itemset_230412: List = []
transactions_tagset_230412: List = []

for threshold in [0.005]:
  for confidence in [0.6, 0.5, 0.4]:
    apriori_rules, transactions_itemset, transactions_tagset = financialApriori.find_frequent_TAGsets(
        transaction_itemset = merge_derivative_news_data_230412["ITEM_SPLIT"],
        transaction_tagset = merge_derivative_news_data_230412["TAG_SPLIT"],
        drop_tagset_list = ["톰슨로이터"],
        minimum_support_threshold = threshold,
        minimum_confidence_threshold = confidence
    )
    apriori_results_230412.append(apriori_rules)
    transactions_itemset_230412.append(transactions_itemset)
    transactions_tagset_230412.append(transactions_tagset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
pd.set_option("display.max_row", 10)
# display( apriori_results_230412[0].sort_values("confidence", ascending = False) )
# display( apriori_results_230412[1].sort_values("confidence", ascending = False) )
display( apriori_results_230412[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(마이크로소프트),(),0.008073,0.097955,0.008073,1.000000,10.208791,0.007282,inf,0.909387
158,(필수소비재),(미국),0.006459,0.138859,0.006459,1.000000,7.201550,0.005562,inf,0.866739
61,(씨티그룹),(미국),0.006459,0.138859,0.006459,1.000000,7.201550,0.005562,inf,0.866739
67,(웰스파고),(제이피모간체이스),0.005920,0.008073,0.005920,1.000000,123.866667,0.005873,inf,0.997834
68,(웰스파고),(미국),0.005920,0.138859,0.005920,1.000000,7.201550,0.005098,inf,0.866270
...,...,...,...,...,...,...,...,...,...,...
161,(미디어),(콘텐츠),0.023681,0.019376,0.009688,0.409091,21.113636,0.009229,1.659518,0.975744
25,(LG전자),(커뮤니케이션),0.017223,0.013455,0.006997,0.406250,30.192500,0.006765,1.661549,0.983823
91,(EU),(금융),0.019914,0.053821,0.008073,0.405405,7.532432,0.007001,1.591301,0.884862
129,(이머징마켓),(금융),0.024220,0.053821,0.009688,0.400000,7.432000,0.008384,1.576964,0.886928


In [28]:
transactions_itemset_230412 = list( itertools.chain.from_iterable(transactions_itemset_230412) )
transactions_tagset_230412 = list( itertools.chain.from_iterable(transactions_tagset_230412) )
print(len(transactions_itemset_230412), len(transactions_tagset_230412))

2670 2124


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


```
support = association_rules_df['support']
confidence = association_rules_df['confidence']

h = 347
s = 1
v = 1
colors = [
    mcl.hsv_to_rgb((h/360, 0.2, v)),
    mcl.hsv_to_rgb((h/360, 0.55, v)),
    mcl.hsv_to_rgb((h/360, 1, v))
]
cmap = LinearSegmentedColormap.from_list('my_cmap',colors,gamma=2)

measures = ['lift', 'leverage', 'conviction',
            'all-confidence', 'collective strength', 'cosine similarity']

fig = plt.figure(figsize=(15,10))
fig.set_facecolor('white')
for i, measure in enumerate(measures):
    ax = fig.add_subplot(320+i+1)
    if measure != 'all-confidence':
        scatter = ax.scatter(support,confidence,c=association_rules_df[measure],cmap=cmap)
    else:
        scatter = ax.scatter(support,confidence,c=association_rules_df['all-confidence'].map(lambda x: [v for k,v in x.items()][0]),cmap=cmap)
    ax.set_xlabel('support')
    ax.set_ylabel('confidence')
    ax.set_title(measure)
    
    fig.colorbar(scatter,ax=ax)
fig.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()
```

## **CS_DATA**와 MARKET_INDEX_NEWS_DATA 8가지 시점별 병합

* SY's Q.
  
  시장 전체에 대한 AU뉴스와 종목과 연관된 AI뉴스를 구별하여 연관규칙 분석을 수행할 필요가 있지 않을까

* 2023.03.15일자는 어떤 일자이기에 AI뉴스의 TAG LIST(해시태그로 추정) 전체가 결측값인걸까

* 시계열 정보 : 4개월간의 자산 및 거래, 접속내역 데이터
* 후위 표시자로 시기 구분
* 자산내역 : 월말일 기준 데이터 제공

  M1(2022.12), M2(2023.01), M3(2023.02), M4(2023.03)
* 거래내역, 접속내역 : 월별 초순, 중순, 하순 동안의 데이터 집계내역 제공

  M2_1(2023.01 초순 1 ~ 10일), M2_2(2023.01 중순 11 ~ 20일), M2_3(2023.01 하순 21 ~ 말일)을 _1, _2, _3으로 구분

* https://chaelist.github.io/docs/network_analysis/network_basics/

* 『전략적 인출 설계와 은퇴 포트폴리오의 과학』


#### **1.필요한 데이터 처리하는 함수 및 전처리된 데이터**

In [29]:
class LoadData():

# 클래스(로컬공간) 내부에서 cs_data, apy_itm_hist_dat, market_index_data의 값
# 생성자를 활용하여 초기화하고 클래스 내부에 캡슐화
  def __init__(self,
               cs_data_path: str, apy_itm_hist_path: str):
    # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
    cs_chunk = pd.read_csv(cs_data_path, chunksize = 10**5, index_col = False, na_values = "NaN", encoding = "utf-8")
    self.cs_data = list(cs_chunk)
    self.apy_itm_hist_data = pd.read_csv(apy_itm_hist_path, index_col = False, na_values = "NaN", encoding = "utf-8")
      # [BAS_YM] (19.12 ~ 23.04) 기준연월
      # [APY_ITM] (19.12 ~ 23.04)간 공모주 청약 진행된 종목 번호
      # [APY_FIN_YM] 이전 3년(19.12 ~ 22.11)간 마지막으로 참여한 공모주 청약연월(X)
      # [APY_ITM_CNT] 이전 3년(19.12 ~ 22.11)간 공모주 청약에 참여한 종목 개수(X)


# parallelDataFrame()
# csv(쉼표로 분할된) 데이터 로드 시 병렬처리하는 메서드
# 실제로 수행할 정도로 데이터 크기가 크지는 않고 100,000행씩 chunking하면 해결됨
  def parallelDataFrame(self, data, function):
    # 코어 수 확인(multiprocessing as mp)
    mp.cpu_count()
    # 코어 수를 전역변수 선언
    global num_cores
    # 코어 수만큼 입력받은 데이터를 2차원 배열로 분할하여 멀티프로세싱(병렬처리)
    data_split = np.array_split(data, num_cores)
    # 코어 수만큼 pool을 생성
    pool = mp.Pool(num_cores)
    # 나누어진 데이터를 function으로 넘겨서 병렬처리 수행
    data = pd.concat(pool.map(
        function, iterable = data_split))
    pool.close()
    # 모두 완료될 때까지 대기
    pool.join
    return data


# loadData()
# 불러온 데이터를 반환(코드 구현 시 활용하고 나중에는 제거할 메서드)
  def loadData(self):
    return self.cs_data, self.apy_itm_hist_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
DataInstance = LoadData(
    cs_data_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/cs_data.csv",
    apy_itm_hist_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/apy_itm_hist.csv",
)
  # [참고] 경로 너무 길어지니까 겹치는 경로는 os.path.join 활용하여 반복 줄일 수 있음

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
consumer_data, apply_item_histogram = DataInstance.loadData()
  # cs_data, apy_item_hist 데이터

# consumer_merge_data = pd.concat(consumer_data)
  # [참고] 텍스트 파일로 저장해도 1.23GB 용량은 줄지 않는바 수행하지 않음
  # cs_merge_data.to_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/cs_data.txt", index = False)

# _, column_length = consumer_merge_data.shape

# 총 552개 컬럼(dtypes: float64(325), int64(2), object(195))으로
# 컬럼별 타입 및 결측값 확인 반복문 수행 필요

# for index in range(0, column_length, 50):
   # consumer_merge_data.iloc[:, index : index + 50].info()

# msno.bar(consumer_merge_data)
  # [참고] 데이터 상세상 0과 결측(NULL)의 구분이 없다는 점에서
  # 함부로 결측값을 예측모델로 대체하면 안됨

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
class GetStockCodeData():

  def __init__(self,
               KOR_SECTOR = None):
    self.KOR_SECTOR = KOR_SECTOR


  def getStockCode(self, date, market):

    stock_list = pd.DataFrame(
        {"종목코드": stock.get_market_ticker_list(date, market = market)})

    stock_list["종목명"] = stock_list["종목코드"].map(lambda x: stock.get_market_ticker_name(x))
    # https://github.com/financedata-org/FinanceDataReader/blob/develop/krx/data.py
    # fdr_data = fdr.StockListing("KRX")
    # stock_list["업종"] = stock_list["종목명"].map(lambda x: fdr_data[fdr_data["Name"] == x]["Sector"].iloc[0])
    return stock_list


  def getStockList(self, base_date):

    self.KOSPI_TICKER = self.getStockCode(date = base_date, market = "KOSPI")
    self.KOSDAQ_TICKER = self.getStockCode(date = base_date, market = "KOSDAQ")


  def getStockSector(self, sector_code_list: List):

    DATA_SECTOR: List = []
    for sector_code in sector_code_list:
        URL = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20230712&sec_cd={ sector_code }'''
        DATA = requests.get(URL).json()
        DATA_JSON = pd.json_normalize(DATA["list"])
        DATA_SECTOR.append(DATA_JSON)
        time.sleep(2)
    KOR_SECTOR = pd.concat(DATA_SECTOR, axis = 0)
    KOR_SECTOR.rename(columns = {"CMP_CD": "TICKER"}, inplace = True)
    self.KOR_SECTOR = KOR_SECTOR[["TICKER", "SEC_NM_KOR"]]

    # sector_code_list = ["G10","G15","G20","G25", "G30", "G35", "G40", "G45","G50", "G55"]


  def getFundamental(self, base_date: str):

    KOSPI_FUNDAMENTAL = stock.get_market_fundamental_by_ticker(
        date = base_date, market = "KOSPI")
    KOSPI_FUNDAMENTAL["종목코드"] = KOSPI_FUNDAMENTAL.index
    KOSPI_FUNDAMENTAL.reset_index(drop = True, inplace = True)
    self.KOSPI_TICKER = pd.merge(
        self.KOSPI_TICKER, KOSPI_FUNDAMENTAL,
        left_on = "종목코드", right_on = "종목코드", how = "outer")
    self.KOSPI_TICKER = pd.merge(
        self.KOSPI_TICKER, self.KOR_SECTOR,
        left_on = "종목코드", right_on = "TICKER", how = "left")

    KOSDAQ_FUNDAMENTAL = stock.get_market_fundamental_by_ticker(
        date = base_date, market = "KOSDAQ")
    KOSDAQ_FUNDAMENTAL["종목코드"] = KOSDAQ_FUNDAMENTAL.index
    KOSDAQ_FUNDAMENTAL.reset_index(drop = True, inplace = True)
    self.KOSDAQ_TICKER = pd.merge(
        self.KOSDAQ_TICKER, KOSDAQ_FUNDAMENTAL,
        left_on = "종목코드", right_on = "종목코드", how = "outer")
    self.KOSDAQ_TICKER = pd.merge(
        self.KOSDAQ_TICKER, self.KOR_SECTOR,
        left_on = "종목코드", right_on = "TICKER", how = "left")

    return self.KOSPI_TICKER, self.KOSDAQ_TICKER


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
Indicator = GetStockCodeData()
# STOCK_CODE_LIST = Indicator.getStockCode()
Indicator.getStockList(base_date = "2023-03-31")
Indicator.getStockSector(sector_code_list = ["G10","G15","G20","G25", "G30", "G35", "G40", "G45","G50", "G55"])
KOSPI_DATA, KOSDAQ_DATA = Indicator.getFundamental(base_date = "2023-03-31")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
STOCK_CODE_LIST = pd.concat(
    [ KOSPI_DATA[["종목코드", "종목명"]], KOSDAQ_DATA[["종목코드", "종목명"]] ],
    axis = 0, ignore_index = True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* 특정 기간 동안의 주식 매수액 1(2, 3)위 종목번호

In [35]:
CS_BUY_M2_3 = pd.DataFrame()
CS_BUY_M3_1 = pd.DataFrame()
CS_BUY_M4_2 = pd.DataFrame()

for i in range(len(consumer_data)):
  CS_BUY_M2_3 = pd.concat(
      [ CS_BUY_M2_3, consumer_data[i].loc[:, ["BUY1_ITM_M2_3", "BUY2_ITM_M2_3", "BUY3_ITM_M2_3"]] ]).dropna()
  CS_BUY_M3_1 = pd.concat(
      [ CS_BUY_M3_1, consumer_data[i].loc[:, ["BUY1_ITM_M3_1", "BUY2_ITM_M3_1", "BUY3_ITM_M3_1"]] ]).dropna()
  CS_BUY_M4_2 = pd.concat(
      [ CS_BUY_M4_2, consumer_data[i].loc[:, ["BUY1_ITM_M4_2", "BUY2_ITM_M4_2", "BUY3_ITM_M4_2"]] ]).dropna()

CS_BUY_M2_3["CONSUMER_ID"] = CS_BUY_M2_3.index
CS_BUY_M3_1["CONSUMER_ID"] = CS_BUY_M3_1.index
CS_BUY_M4_2["CONSUMER_ID"] = CS_BUY_M4_2.index

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* 특정 기간 동안의 주식 매도액 1(2, 3)위 종목번호


In [36]:
CS_SELL_M2_3 = pd.DataFrame()
CS_SELL_M3_1 = pd.DataFrame()
CS_SELL_M4_2 = pd.DataFrame()

for i in range(len(consumer_data)):
  CS_SELL_M2_3 = pd.concat(
      [ CS_SELL_M2_3, consumer_data[i].loc[:, ["SEL1_ITM_M2_3", "SEL2_ITM_M2_3", "SEL3_ITM_M2_3"]] ]).dropna()
  CS_SELL_M3_1 = pd.concat(
      [ CS_SELL_M3_1, consumer_data[i].loc[:, ["SEL1_ITM_M3_1", "SEL2_ITM_M3_1", "SEL3_ITM_M3_1"]] ]).dropna()
  CS_SELL_M4_2 = pd.concat(
      [ CS_SELL_M4_2, consumer_data[i].loc[:, ["SEL1_ITM_M4_2", "SEL2_ITM_M4_2", "SEL3_ITM_M4_2"]] ]).dropna()
# Counter(CS_SELL_M2_3["SEL1_ITM_M2_3"])

CS_SELL_M2_3["CONSUMER_ID"] = CS_SELL_M2_3.index
CS_SELL_M3_1["CONSUMER_ID"] = CS_SELL_M3_1.index
CS_SELL_M4_2["CONSUMER_ID"] = CS_SELL_M4_2.index

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* AI주식종목 데이터 연관규칙분석 결과(왼쪽 빈발집합, 오른쪽 빈발집합, 신뢰도)

In [37]:
APRIORI_NEWS_230125 = apriori_results_230125[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]
APRIORI_NEWS_230126 = apriori_results_230126[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]
APRIORI_NEWS_230127 = apriori_results_230127[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]
APRIORI_NEWS_230131 = apriori_results_230131[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]
APRIORI_NEWS_230201 = apriori_results_230201[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]
APRIORI_NEWS_230202 = apriori_results_230202[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]
APRIORI_NEWS_230310 = apriori_results_230310[2].sort_values("confidence", ascending = False)[["antecedents", "consequents", "confidence"]]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* 엑셀의 VLOOKUP과 같은 기능의 익명함수를 만들 것

```
    for column in CS_BUY_M2_3.columns:
      CS_BUY_M2_3[column] = CS_BUY_M2_3[column].apply(
          lambda ticker:
          STOCK_CODE_LIST["종목명"] if ticker == STOCK_CODE_LIST["종목코드"] else ticker)
```

___

#### **2.연관규칙 분석 결과 기반의 이원 네크워크 분석**

In [93]:
class NetworkX():

  def __init__(self, CS_BUY_DATA, CS_SELL_DATA,
               STOCK_CODE_LIST, APRIORI_NEWS_DATA,
               CS_BUY_LIST = None, CS_SELL_LIST = None):

    self.CS_BUY_DATA = CS_BUY_DATA
    self.CS_SELL_DATA = CS_SELL_DATA
    self.STOCK_CODE_LIST = STOCK_CODE_LIST
    self.APRIORI_NEWS_DATA = APRIORI_NEWS_DATA

    self.CS_BUY_LIST = CS_BUY_LIST
    self.CS_SELL_LIST = CS_SELL_LIST


  def getItemsToBuy(self):

    CS_BUY_LIST: List = []
    for column in self.CS_BUY_DATA.drop(["CONSUMER_ID"], axis = 1).columns:
      CS_BUY_MERGE = pd.merge(
          self.CS_BUY_DATA[["CONSUMER_ID", column]], STOCK_CODE_LIST,
          left_on = column, right_on = "종목코드", how = "inner")
      CS_BUY_LIST.append(CS_BUY_MERGE)
    self.CS_BUY_LIST = CS_BUY_LIST
    return self.CS_BUY_LIST


  def getItemsToSell(self):

    CS_SELL_LIST: List = []
    for column in self.CS_SELL_DATA.drop(["CONSUMER_ID"], axis = 1).columns:
      CS_SELL_MERGE = pd.merge(
          self.CS_SELL_DATA[["CONSUMER_ID", column]], STOCK_CODE_LIST,
          left_on = column, right_on = "종목코드", how = "inner")
      CS_SELL_LIST.append(CS_SELL_MERGE)
    self.CS_SELL_LIST = CS_SELL_LIST
    return self.CS_SELL_LIST


  def createDirectedBipartiteGraphFromAprioriResults(self,
                                                     threshold_confidence: float,
                                                     RANK: int):
  # 방향이 있는 이분 그래프를 생성
    direction_graph = nx.DiGraph()

  # 뉴스태그와 뉴스종목들을 첫 번째 집합으로 추가
    NODES_SET1 = set( self.APRIORI_NEWS_DATA["antecedents"].explode().unique() )
    NODES_SET2 = set( self.APRIORI_NEWS_DATA["consequents"].explode().unique() )
    direction_graph.add_nodes_from(NODES_SET1, bipartite = 0)
    direction_graph.add_nodes_from(NODES_SET2, bipartite = 0)

  # 소비자의 매수 종목을 두 번째 집합으로 추가(고객ID는 어떻게 해야할까)
    direction_graph.add_nodes_from(
        self.CS_BUY_LIST[RANK - 1]["종목명"], bipartite = 1)

  # 각 연관 규칙에 대한 confidence 값을 이용하여 간선의 방향 결정
    for _, row in self.APRIORI_NEWS_DATA.iterrows():
      antecedents = set( row["antecedents"] )
      consequents = set( row["consequents"] )
      confidence = row["confidence"]

    # confidence 값이 threshold 이상인 경우에만 엣지(간선) 추가
      if confidence >= threshold_confidence:
        for antecedent in antecedents:
          for consequent in consequents:
            direction_graph.add_edge(antecedent, consequent)

    # 소비자의 매수 종목과 뉴스종목 간의 엣지(간선) 추가
    for cs_ticker in self.CS_BUY_LIST[RANK - 1]:
      for news_stock in NODES_SET2:
          if stock == cs_ticker:
            direction_graph.add_edge(cs_ticker, news_stock)

    # 뉴스종목과 소비자의 매수 종목이 중복되는 경우
    # 뉴스종목에서 소비자의 매수 종목으로 방향이 있는 간선 추가
    for _, row in self.APRIORI_NEWS_DATA.iterrows():
      consequents = set( row["consequents"] )

      for cs_ticker in self.CS_BUY_LIST[RANK - 1]["종목명"].tolist():
        # TypeError: unhashable type: 'DataFrame'
        if news_stock in consequents:
          for antecedent in row["antecedents"]:
            direction_graph.add_edge(antecedent, news_stock)

    return direction_graph


  def visualizeBipartiteGraph(self, graph, nanum_font_path):
    plt.figure(figsize = (50, 15))

    font_path = nanum_font_path
    font_name = fm.FontProperties(fname = font_path, size = 10).get_name()
    plt.rc("font", family = font_name)
    # plt.rcParams["font.family"] = font_name

    pos = nx.random_layout(graph)
      # 레이아웃 알고리즘을 spring_layout로 설정
    node_color_map = { node: "skyblue" for node in graph.nodes() }

    for node in graph.nodes():

  # 첫 번째 집합 (뉴스태그와 뉴스종목)
      if "bipartite" in graph.nodes[node] and graph.nodes[node]["bipartite"] == 0:
            node_color_map[node] = "lightgreen"
  # 두 번째 집합 (소비자의 매수 종목)
      else: node_color_map[node] = 'lightpink'

    nx.draw(graph,
            pos,
            with_labels = True,
            labels = { node: node for node in graph.nodes() },
            node_color = [node_color_map[node] for node in graph.nodes()], node_size = 500,
            font_size = 20, font_weight = "bold",
            arrowsize = 20, arrowstyle = '->')
    plt.show()

    # font_path = "/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf"


  def KernighanAlgoCommunity(self, graph):
    from networkx.algorithms.community import kernighan_lin_bisection

    graph_undirected_bipartite = graph.to_undirected()
    communities = kernighan_lin_bisection(graph_undirected_bipartite)
    for community in communities:
      print(community)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
AprioriNetwork =  NetworkX(
    CS_BUY_DATA = CS_BUY_M2_3,
    CS_SELL_DATA = CS_SELL_M2_3,
    STOCK_CODE_LIST = STOCK_CODE_LIST,
    APRIORI_NEWS_DATA = APRIORI_NEWS_230125,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



소비자 매도매수 종목과 장바구니 분석결과를 바탕으로 유향그래프를 그리고 강한 연결 요소를 탐지하는 방법을 설명드리겠습니다. 이때 소비자 매도매수 종목에 색깔 같은 특징을 부여하여 강한 연결 내에서 소비자 매수매도 종목이 얼마나 포함되어 있는지를 파악할 수 있습니다.

• 데이터 전처리: 소비자 매도매수 종목과 장바구니 분석결과를 바탕으로 유향그래프를 생성하기 위해 필요한 데이터를 전처리합니다. 소비자 매도매수 종목과 장바구니 분석결과를 정리하고, confidence 값을 크기에 따라 정렬하여 장바구니 분석결과와 종목들 사이의 간선을 생성합니다.


In [95]:
CS_BUY_LIST = AprioriNetwork.getItemsToBuy()
CS_SELL_LIST = AprioriNetwork.getItemsToSell()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


• 유향그래프 생성: 전처리한 데이터를 바탕으로 유향그래프를 생성합니다. 소비자 매도매수 종목과 장바구니 분석결과를 노드로, confidence 값을 가중치로 하는 간선을 이용하여 그래프를 생성합니다.

• 강한 연결 요소 탐지: 생성된 유향그래프에서 카잔 알고리즘 등을 사용하여 강한 연결 요소를 탐지합니다.

In [96]:
graph_object = AprioriNetwork.createDirectedBipartiteGraphFromAprioriResults(
    threshold_confidence = 0.6,
    RANK = 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


```
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt -qq -y install fonts-nanum
```

In [97]:
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
# [(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name]

matplotlib.rcParams["axes.unicode_minus"] = False
print ('버전: ', matplotlib.__version__)
print ('설치 위치: ', matplotlib.__file__)
print ('설정 위치: ', matplotlib.get_configdir())
print ('캐시 위치: ', matplotlib.get_cachedir())
print ('설정파일 위치: ', matplotlib.matplotlib_fname())

버전:  3.7.1
설치 위치:  /usr/local/lib/python3.10/dist-packages/matplotlib/__init__.py
설정 위치:  /root/.config/matplotlib
캐시 위치:  /root/.cache/matplotlib
설정파일 위치:  /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/matplotlibrc


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
# fm._rebuild()
  # 과거 샘플코드와 현재 라이브러리 버전 호환이 안되는 것
# fm.get_fontconfig_fonts()

font_list = fm.findSystemFonts(
    fontpaths = None, fontext = "ttf")
print(font_list, end = ", ")

['/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf', '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf', '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf', '/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf', '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf', '/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf', '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf', '/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf', '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf', '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf', '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', '/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', '/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf', '/usr/share/fonts/truetype/liberation/LiberationMono-Italic.ttf', '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf', '/usr/share/fonts/truetyp

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


• 그래프 시각화: 최종적으로 생성된 유향그래프를 시각화하여 분석 결과를 확인합니다. 이때, 색깔 특징을 부여한 노드들이 어떻게 연결되어 있는지를 살펴봄으로써 소비자 매수매도 종목과 장바구니 분석결과 간의 강한 연결성과 중요성을 시각적으로 확인할 수 있습니다.

위와 같은 방법을 사용하여 소비자 매수매도 종목과 장바구니 분석결과를 연결하는 유향그래프를 생성하고, 강한 연결 요소를 찾아내며, 소비자 매수매도 종목에 색깔 특징을 부여하여 시각화함으로써 소비자의 투자에 대한 정보를 더욱 상세하게 분석할 수 있습니다. 이를 통해 소비자의 투자지식에 따른 포트폴리오 모델링과 투자심리지수 분석에 도움을 줄 수 있습니다.

• 색깔 특징 부여: 강한 연결 요소를 탐지한 후, 해당 요소 내의 노드들 중 소비자 매도매수 종목들에 색깔을 부여합니다. 예를 들어, 강한 연결 요소 내의 노드들은 같은 색으로 표시하고, 소비자 매수매도 종목들은 다른 색으로 표시합니다.

In [ ]:
AprioriNetwork.visualizeBipartiteGraph(
    graph = graph_object,
    nanum_font_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/NanumSquareRoundR.ttf"
    # nanum_font_path = "C:/WINDOWS/FONTS/NANUMGOTHIC.TTF"
    # nanum_font_path = "/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf"
)

```
plt.figure(figsize = (50, 15))

font_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/NanumSquareRoundR.ttf"
font_name = fm.FontProperties(fname = font_path, size = 10).get_name()
plt.rc("font", family = font_name)
# plt.rcParams["font.family"] = font_name

pos = nx.random_layout(graph_object)  
  # 레이아웃 알고리즘을 spring_layout로 설정
node_color_map = { node: "skyblue" for node in graph_object.nodes() }
    
for node in graph_object.nodes():
  
# 첫 번째 집합 (뉴스태그와 뉴스종목)
  if "bipartite" in graph_object.nodes[node] and graph_object.nodes[node]["bipartite"] == 0:
    node_color_map[node] = "lightgreen"
  # 두 번째 집합 (소비자의 매수 종목)  
  else: node_color_map[node] = 'lightpink'

  nx.draw(
      graph_object,
      pos,
      with_labels = True,
      labels = { node: node for node in graph_object.nodes() },
      node_color = [node_color_map[node] for node in graph_object.nodes()], node_size = 500,
      font_size = 20, font_weight = "bold",
      arrowsize = 20, arrowstyle = '->')
plt.show()
```


* 카잔 알고리즘

  • 그래프의 각 노드를 방문하지 않은 상태로 표시합니다.

  • 각 노드에 대해 깊이 우선 탐색(DFS)을 수행합니다. 이 때, DFS를 수행하면서 방문한 노드들을 스택에 추가합니다.

  • DFS를 모든 노드에 대해 수행하면서 스택에 추가된 노드들을 역순으로 꺼내면서 역방향 그래프(각 간선의 방향을 반대로 한 그래프)를 생성합니다.

  • 역방향 그래프에 대해 다시 한 번 DFS를 수행하면서 강한 연결 요소를 찾아냅니다.

In [ ]:
AprioriNetwork.KernighanAlgoCommunity(graph = graph_object)

* 연관규칙분석을 수행한 결과를 바탕으로 노드와 엣지를 구성할 데이터 전처리
* 노드 : 연관규칙 항목들을 노드로 사용
* 엣지 : 항목 간의 연관성 정보(신뢰도) -> 엣지의 가중치 할당

* 링크의 방향성이 없는 네트워크 형성(해당 변수명에 속성 부여)
* 관계에서 두 node가 갖는 특성이 비슷한 경우(undirexted tie)
```
graph = nx.DiGraph()
```

* 링크의 방향성이 있는 네트워크 형성
* 관계에서 두 node가 갖는 특성이 다른 경우
```
direction_graph = nx.DiGraph()
```